# Started from here!

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

import cv2
import numpy as np
from PIL import Image
import os

import heapq
import itertools
from sklearn.decomposition import PCA

In [2]:
test_dir = 'data/merged/test/'
train_dir = 'data/merged/train/'

#b_train_dir = 'data/balanced/'

classes = os.listdir(train_dir)
test_cls = os.listdir(train_dir)
num_classes = len(classes)

In [3]:
print(
    f'Number of classes: {num_classes}',
    f'\nClasses: {classes}'
)

Number of classes: 7 
Classes: ['fear', 'angry', 'sad', 'neutral', 'surprise', 'disgust', 'happy']


In [4]:
print("--------Train--------")
# check the number of images in each class
for cls in classes:
    print(f'{cls}: {len(os.listdir(train_dir + cls))}')

print("\n--------Test--------")
# check the number of images in each class
for cls in classes:
    print(f'{cls}: {len(os.listdir(test_dir + cls))}')
    
#print("\n--------balance--------")
## check the number of images in each class
#for cls in classes:
#    print(f'{cls}: {len(os.listdir(b_train_dir + cls))}')

--------Train--------
fear: 5000
angry: 5000
sad: 5000
neutral: 5000
surprise: 5000
disgust: 5000
happy: 7215

--------Test--------
fear: 1024
angry: 1000
sad: 1247
neutral: 1233
surprise: 1000
disgust: 1000
happy: 1774


In [5]:
import os
import imghdr
import shutil

def delete_irrelevant_files(directory, classes):
    for cls in classes:
        class_dir = os.path.join(directory, cls)
        with os.scandir(class_dir) as entries:
            for entry in entries:
                # Delete directories
                if entry.is_dir():
                    print(f'Deleting directory: {entry.path}')
                    shutil.rmtree(entry.path)

                # Check if the file is empty and delete it
                elif os.path.getsize(entry.path) == 0:
                    print(f'Deleting empty file: {entry.path}')
                    os.remove(entry.path)

                # Check if the file is not an image and delete it
                elif not imghdr.what(entry.path):
                    print(f'Deleting non-image file: {entry.path}')
                    os.remove(entry.path)

print("--------Train--------")
delete_irrelevant_files(train_dir, classes)

print("\n--------Test--------")
delete_irrelevant_files(test_dir, classes)

#delete_irrelevant_files(b_train_dir, classes)


--------Train--------

--------Test--------


# Landmark detection from images and correlation of those landmark positions

In [6]:
image_size = torch.Size([48, 48, 3])
batch_size = 256

# Compute the total size of the image input
image_input_size = np.prod(image_size) * batch_size * 4. / (1024 ** 2.)
print(f"Image input size: {image_input_size:.2f} MB")

Image input size: 6.75 MB


In [7]:
class FER_Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []  # new list to store the label index
        for i, label in enumerate(os.listdir(root_dir)):
            class_dir = os.path.join(root_dir, label)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                image = cv2.imread(img_path)
                self.data.append(image)
                self.labels.append(i)  # add the label index for this image

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]  # get the label index for this image

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, label  # return the image and label index

In [8]:
# Train the model
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = FER_Dataset(train_dir, transform=transform)
val_dataset = FER_Dataset(test_dir, transform=transform)

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=24)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=24)

In [10]:
# show shape of the train_dataloader
for i, data in enumerate(train_dataloader):
    print(i, data[0].shape, data[1].shape)
    break

# show shape of the val_dataloader
for i, data in enumerate(val_dataloader):
    print(i, data[0].shape, data[1].shape)
    break

# torch.Size([4, 3, 48, 48]) torch.Size([4])
# torch.Size([4, 3, 48, 48]) torch.Size([4])

# 0 torch.Size([256, 3, 64, 64]) torch.Size([256])
# 0 torch.Size([256, 3, 64, 64]) torch.Size([256])

0 torch.Size([256, 3, 128, 128]) torch.Size([256])
0 torch.Size([256, 3, 128, 128]) torch.Size([256])


In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.input_dim = input_dim
        self.num_heads = num_heads
        self.head_dim = input_dim // num_heads

        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)

        self.fc = nn.Linear(input_dim, input_dim)

    def scaled_dot_product_attention(self, query, key, value):
        qk = torch.matmul(query, key.transpose(-2, -1))
        dk = query.size(-1)
        scaled_attention_logits = qk / torch.sqrt(torch.tensor(dk, dtype=torch.float32, device=qk.device))

        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.num_heads, self.head_dim)
        return x.transpose(1, 2)

    def forward(self, x):
        batch_size = x.size(0)

        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        query = self.split_heads(query)
        key = self.split_heads(key)
        value = self.split_heads(value)

        out, attention_weights = self.scaled_dot_product_attention(query, key, value)
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.input_dim)

        out = self.fc(out)
        
        return out

In [2]:
!pip install yolov5

  Using cached yolov5-7.0.11-py37.py38.py39.py310-none-any.whl (956 kB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 4.3 MB/s eta 0:00:00
  Using cached roboflow-1.0.1-py3-none-any.whl (55 kB)
  Using cached fire-0.5.0.tar.gz (88 kB)
  Preparing metadata (setup.py) ... done
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached requests_toolbelt-0.10.1-py2.py3-none-any.whl (54 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached pybboxes-0.1.6-py3-none-any.whl (24 kB)
  Using cached terminaltables-3.1.10-py2.py3-none-any.whl (15 kB)
  Using cached click-8.0.4-py3-none-any.whl (97 kB)
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116931 sha256=eda09413ff7fcc8338ccd4ad6cebbfd9ce6dddaaff65b5d4276ccbffb87628f8
  Stored in directory

In [1]:
from yolov5.models.yolo import Model

def load_pretrained_yolov5(path: str, num_classes: int):
    checkpoint = torch.load(path)
    model = Model(checkpoint['model'].yaml)
    model.load_state_dict(checkpoint['model'].state_dict())

    for name, param in model.named_parameters():
        # Freeze all layers except the last three
        if not any(name.startswith(f"model.{i}") for i in [17, 20, 23]):
            param.requires_grad = False

    # Replace the last layer with a new one, adjust the number of classes
    model.model[-1].nc = num_classes  # Set the number of classes in the Detect layer

    # Reinitialize the last layer's weights
    for m in model.modules():
        if type(m) is nn.Conv2d:
            torch.nn.init.normal_(m.weight, std=0.01)
            if m.bias is not None:
                torch.nn.init.constant_(m.bias, 0)

    return model

ModuleNotFoundError: No module named 'yolov5'

In [14]:
class FacialExpressionDetectionModel(nn.Module):
    def __init__(self, num_classes, num_heads):
        super(FacialExpressionDetectionModel, self).__init__()
        self.num_classes = num_classes
        # Function to load the pretrained YOLOv5 model
        self.yolov5 = load_pretrained_yolov5(path="yolov5s.pt",num_classes=num_classes)
        self.yolov5 = nn.Sequential(*list(self.yolov5.children())[:-1])  # Remove the Detect layer

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 256, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 128, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.5)
        )

        # Block-2
        self.conv2 = nn.Sequential(
            nn.Conv2d(128, 512, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 256, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.5)
        )

        # Block-3
        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 128, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )

        # Block-4
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 256, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.125)
        )

        # Block-5
        self.fc1 = nn.Sequential(
            nn.Linear(256 * 8 * 8, 512),  # Update the input size to match the new input shape
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25)
        )

        # Attention Layer
        self.attention = MultiHeadAttention(512, num_heads)
        self.attention_transform = nn.Linear(512, 64)

        # Block-6
        self.fc2 = nn.Sequential(
            nn.Linear(64, 32),  # Update the input size to 64
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(32),
            nn.Dropout(0.125),
        )

        # Block-7
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.yolov5(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.attention(x)
        x = x.view(x.size(0), -1)  # Flatten the output of the attention layer
        x = self.attention_transform(x)
        
        x = self.fc2(x)
        x = self.fc3(x)
        
        return F.softmax(x, dim=1)


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [16]:
num_classes = 7
num_heads = 32
model = FacialExpressionDetectionModel(num_classes, num_heads).to(device)


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]              
  8                -1  1   1182720  

In [17]:
with open('FER_model.txt', 'w') as f:
    f.write(str(model))

In [18]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for data, targets in dataloader:
        data = data.to(device)
        targets = targets.to(device)

        # Use autocast for mixed precision
        with autocast():
            outputs = model(data)
            loss = criterion(outputs, targets)
            _, preds = torch.max(outputs, 1)

        # Scale the loss and perform backpropagation
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == targets.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)

    return epoch_loss, epoch_acc

In [19]:
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for data, targets in dataloader:
            data, targets = data.to(device), targets.to(device)

            # Use autocast for mixed precision
            with autocast():
                outputs = model(data)
                loss = criterion(outputs, targets)
                _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * data.size(0)
            running_corrects += torch.sum(preds == targets.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)

    return epoch_loss, epoch_acc

In [20]:
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = FacialExpressionDetectionModel(num_classes, num_heads).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, threshold=0.0001)
scaler = GradScaler()


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]              
  8                -1  1   1182720  

In [21]:
print(device)

cuda


In [22]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
import matplotlib.pyplot as plt

# create empty lists to store loss and accuracy for each epoch
train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

num_epochs = 200
for epoch in range(num_epochs):
    
    train_loss, train_acc = train_epoch(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_acc = validate_epoch(model, val_dataloader, criterion, device)
    scheduler.step(val_loss)
    
    # append the loss and accuracy to the lists
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)


    print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch: 1/200, Train Loss: 1.7807, Train Acc: 0.3845, Val Loss: 1.6926, Val Acc: 0.4720


In [ ]:
import torch
import torch.onnx

device = next(model.parameters()).device
dummy_input = torch.randn(1, 3, 128, 128).to(device)

onnx_filename = "FER_Model_Adam.onnx"
torch.onnx.export(model, dummy_input, onnx_filename)

print(f"Model saved as {onnx_filename}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

train_acc_list = [float(t) for t in train_acc_list]
val_acc_list = [float(t) for t in val_acc_list]

train_acc_df = pd.DataFrame({'Epoch': range(len(train_acc_list)), 'Accuracy': train_acc_list, 'Type': 'Train'})
val_acc_df = pd.DataFrame({'Epoch': range(len(val_acc_list)), 'Accuracy': val_acc_list, 'Type': 'Validation'})

acc_df = pd.concat([train_acc_df, val_acc_df])

sns.set_style('darkgrid')

plt.figure(figsize=(10, 6))
sns.lineplot(data=acc_df, x='Epoch', y='Accuracy', hue='Type')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Save the graph to a file
plt.savefig('accuracy_graph_Adam.png', dpi=300)

plt.show()


In [ ]:
train_loss_list = [float(t) for t in train_loss_list]
val_loss_list = [float(t) for t in val_loss_list]

train_loss_list = pd.DataFrame({'Epoch': range(len(train_loss_list)), 'Loss': train_loss_list, 'Type': 'Train'})
val_loss_list = pd.DataFrame({'Epoch': range(len(val_loss_list)), 'Loss': val_loss_list, 'Type': 'Validation'})

acc_df = pd.concat([train_loss_list, val_loss_list])

sns.set_style('darkgrid')

plt.figure(figsize=(10, 6))
sns.lineplot(data=acc_df, x='Epoch', y='Loss', hue='Type')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Save the graph to a file
plt.savefig('loss_graph_Adam.png', dpi=300)

plt.show()


In [ ]:
# define a function to check pytorch version
def check_pytorch_version():
    if torch.__version__ >= '1.6.0':
        return True
    else:
        return False
    
# define a function to check cuda version
def check_cuda_version():
    if torch.cuda.is_available():
        return True
    else:
        return False
    
# define a function to check cudnn version
def check_cudnn_version():
    if check_cuda_version():
        if torch.backends.cudnn.enabled:
            return True
        else:
            return False
    else:
        return False
    
# define a function to check if the system is ready for training
def check_system():
    if check_pytorch_version():
        print('PyTorch version: {}'.format(torch.__version__))
    else:
        print('PyTorch version: {} (update required)'.format(torch.__version__))
        
    if check_cuda_version():
        print('CUDA version: {}'.format(torch.version.cuda))
    else:
        print('CUDA version: {} (install CUDA to enable GPU training)'.format(torch.version.cuda))
        
    if check_cudnn_version():
        print('cuDNN version: {}'.format(torch.backends.cudnn.version()))
    else:
        print('cuDNN version: {} (install cuDNN to enable GPU training)'.format(torch.backends.cudnn.version()))

check_system()